## SDXL Resource Lists

**Homepage :** [SD Webui Complete setup](https://github.com/ffxvs/sd-webui-complete-setup)  
**Guide :** [Resource Lists Guide](https://github.com/ffxvs/sd-webui-complete-setup/wiki/Resource-Lists-Guide)

## 1. Requirements

### • Initialization
**REQUIRED EVERY TIME YOU START THE MACHINE**

In [ ]:
# CivitAI Token - Required to download models that require CivitAI login
civitai_token = ''


#########################################################################################################

import os
os.chdir('/')
from internal.main import *

apply_envs1()
completed_message()

## 2. ControlNet

### • ControlNet Models

In [ ]:
# ControlNet - https://github.com/Mikubill/sd-webui-controlnet/discussions/2039
# Select models to download. Default models are : depth, ip-adapter, openpose

controlnet = {  
    'blur': boolean[0],
    'blur_anime': boolean[0],
    'canny': boolean[0],
    'canny_anime': boolean[0],
    'depth': boolean[1],
    'depth_anime': boolean[0],
    'ipadapter': boolean[1],
    'ipadapter_vit_h': boolean[0],
    'ipadapter_plus': boolean[0],
    'ipadapter_plus_face': boolean[0],
    'lineart': boolean[0],
    'openpose': boolean[1],
    'openpose_anime': boolean[0],
    'recolor': boolean[0],
    'scribble_anime': boolean[0],
    'sketch': boolean[0],
    'softedge': boolean[0],
}


#########################################################################################################

print("\n⏳ Download selected controlNet models...")
download_controlnet(controlnet, sdxl_controlnet_url, base.SDXL)
os.chdir(webui_path)
completed_message()

### • Install from URLs

In [ ]:
# Install controlNet models from URL 
other_controlnet = []


#########################################################################################################

if other_controlnet:
    print('⏳ Downloading controlNet models...')
    download_other_res(other_controlnet, f'{controlnet_models_path}/sdxl', civitai_token)

os.chdir(webui_path)
completed_message()

## 3. SDXL Models
**Run the cell and select the model version you want to download**  
Some models hosted by CivitAI require you to login to download.  
Create an API key in [CivitAI user settings](https://civitai.com/user/account) then paste it to `civiai_token` inside Initialization cell.

### • Anime / Cartoon / 3D

In [ ]:
models_selection(sdxl_anime_models_url, base.SDXL, civitai_token)
os.chdir(webui_path)


### • General Purpose

In [ ]:
models_selection(sdxl_general_models_url, base.SDXL, civitai_token)
os.chdir(webui_path)


### • Realistic

In [ ]:
models_selection(sdxl_realistic_models_url, base.SDXL, civitai_token)
os.chdir(webui_path)


### • Woman

In [ ]:
models_selection(sdxl_woman_models_url, base.SDXL, civitai_token)
os.chdir(webui_path)


### • Install from URLs

In [ ]:
# Install models from URL 
other_models = []


#########################################################################################################

if other_models:
    print('⏳ Downloading models...')
    download_other_res(other_models, f'{models_path}/sdxl', civitai_token)

os.chdir(webui_path)
completed_message()

## 4. LoRA, Embedding, Upscaler and VAE

### • Built-in Resources
This is an information cell.  
To download these resources, run the `Resource Lists` cell below.

**Built-in LoRA :**
* [Detail Tweaker XL](https://civitai.com/models/122359)

**Built-in Embeddings :**
* [Negative Prompts Pack XL](https://huggingface.co/ffxvs/negative-prompts-pack-xl)
* [Embeddings Collection XL](https://huggingface.co/ffxvs/embeddings-collection-xl)

**Built-in Upscaler :**  
* [UltraSharp](https://openmodeldb.info/models/4x-UltraSharp)

**Built-in VAE :**
* [sdxl-vae-fp16-fix](https://huggingface.co/madebyollin/sdxl-vae-fp16-fix)

### • Resource Lists

In [ ]:
# Run this cell to display the resource lists

resources_selection(sdxl_builtin_resources_url, sdxl_resources_url, base.SDXL, civitai_token)
os.chdir(webui_path)

### • Install from URLs

In [ ]:
# LoRA
other.lora = []

# Embeddings
other.embeddings = []

# Upscaler
other.upscaler = []

# VAE
other.vae = []


#########################################################################################################

other_resources(other, base.SDXL, civitai_token)
os.chdir(webui_path)
completed_message()